In [1]:
!pip install -qq scrapy
!pip install langchain
!pip install -qU langchain-community faiss-cpu
!pip install -qU langchain-openai
!pip install --upgrade --quiet  rank_bm25
!pip install langchain_experimental
!pip install -U FlagEmbedding
!pip install --upgrade --quiet  langchain-google-genai
!pip install pyngrok
!pip install flask_cors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

ngrok_token = os.getenv('NGROK_TOKEN')
gemini_token = os.getenv('GEMINI_TOKEN')

# Data Crawling

In [2]:
import requests
import xml.etree.ElementTree as ET
import json

# URLs of the sitemaps
sitemap_urls = [
    'https://www.tiemtraannhien.vn/product-sitemap.xml'
]

all_urls = []

# Function to fetch and parse XML
def fetch_sitemap(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        root = ET.fromstring(response.content)
        # Extract all <loc> elements that contain the URLs
        for url_element in root.iter('{http://www.sitemaps.org/schemas/sitemap/0.9}loc'):
            all_urls.append(url_element.text)
    except Exception as e:
        print(f"Error fetching or parsing {url}: {e}")

# Fetch and parse both sitemaps
for sitemap_url in sitemap_urls:
    fetch_sitemap(sitemap_url)

# Write the URLs to a JSON file
with open('all_urls.json', 'w') as f:
    json.dump(all_urls, f, indent=4)

print(f"Extracted {len(all_urls)} URLs and saved to all_urls.json")


Extracted 47 URLs and saved to all_urls.json


In [3]:
import pandas as pd
import json
import re
# Specify the filename
filename = '/content/all_urls.json'

# Load the all_urls list from the JSON file
with open(filename, 'r') as file:
    all_urls = json.load(file)

print(f"All URLs loaded from {filename}")


import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup

class CustomSpider(scrapy.Spider):
    name = 'custom_spider'
    start_urls = all_urls

    # Initialize a counter
    request_count = 0

    def parse(self, response):
        self.request_count += 1  # Increment the counter with each request
        description = ""

        # Scraping the review title (h1 tag inside div.product_title)
        review_title = response.css('h1.product-title::text').get()

        if review_title:
            h1_tag = review_title.strip()
        else:
            h1_tag = ""

        # Now, h1_tag contains the content of the h1 tag
        print(h1_tag)


        price = response.css('span.woocommerce-Price-amount')

        if price:
            price = price.get().strip()
        else:
            price = ""
        match = re.search(r'>([\d.,]+₫)<', price)
        if match:
            price = match.group(1)
        # Scraping the ck-content
        ck_contents = response.css('div.woocommerce-Tabs-panel--description')

        for ck_content in ck_contents:
            for element in ck_content.xpath('./*'):
                # Extract the text from h2 and h3 tags
                if element.root.tag == 'h2':
                    description += ' '.join(element.css('::text').getall()).strip() + "\n"
                elif element.root.tag == 'h3':
                    description += ' '.join(element.css('::text').getall()).strip() + "\n"

                # Extract the text from p tags
                elif element.root.tag == 'p':
                    description += ' '.join(element.css('::text').getall()).strip() + "\n"

                # Extract the list items from ul tags
                elif element.root.tag == 'ul':
                    li_tags = element.css('li')
                    for li_tag in li_tags:
                        description += f"- {' '.join(li_tag.css('::text').getall()).strip()}" + "\n"

        # Initialize an empty array to hold image URLs
        image_urls = []

        # Select all the div elements with the specific class
        image_elements = response.css('div.woocommerce-product-gallery__image')

        # Loop through each image element to extract the URLs
        for element in image_elements:
            # Extract the main image URL from the 'data-large_image' attribute
            image_url = element.css('img::attr(data-large_image)').get()

            # Add the extracted image URL to the array
            if image_url:
                image_urls.append(image_url)

        data = {}

        if h1_tag and description:
            data = {
                "url": response.url,  # Add the URL of the request
                "content": description,
                "price": price,
                "title": h1_tag,  # h1_tag is now guaranteed to be a string
                "image_urls": image_urls
            }

            yield data
        # Print out the current request count
        print('====> h1_tag', h1_tag)
        print('====>description', description)
        print('====>image_urls', image_urls)
        print('====>price', price)

        self.logger.info(f"Number of requests done: {self.request_count}")
        self.logger.info(f"Crawled: {response.url}")


# Initialize the Scrapy crawler process
process = CrawlerProcess({
    'LOG_LEVEL': 'INFO',
    'FEEDS': {
        'output.json': {
            'format': 'json',
            'encoding': 'utf8',
            'store_empty': False,
            'fields': None,
            'indent': 4,
        },
    },
    'CLOSESPIDER_TIMEOUT': 60000000000,  # Close the spider after 60 seconds (adjust as needed)
    'DOWNLOAD_DELAY': 3,  # Delay of 2 seconds between each request
})

# Start the spider
process.crawl(CustomSpider)
process.start()


All URLs loaded from /content/all_urls.json


INFO:scrapy.utils.log:Scrapy 2.12.0 started (bot: scrapybot)
2025-03-24 14:06:14 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-03-24 14:06:14 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2025-03-24 14:06:14 [scrapy.addons] INFO: Enabled addons:
[]
DEBUG:scrapy.utils.log:Using reactor: twisted.internet.epollreactor.EPollReactor
INFO:scrapy.extensions.telnet:Telnet Password: e088fc0c65e6269d


Củ Sen Sấy Giòn
====> h1_tag Củ Sen Sấy Giòn
====>description CỦ SEN SẤY GIÒN – TRỌN VỊ NGON NÔNG SẢN VIỆT
Tiệm trà An Nhiên chuyên cung cấp các sản phẩm chăm sóc sắc đẹp và sức khỏe có thành phần từ thiên nhiên với công nghệ sản xuất khác biệt, trong đó có thể kể đến các sản phẩm như trà dưỡng tâm an thần, trà gừng đường nâu thảo mộc, trà dưỡng nhan, trà gạo lứt đậu đen, bột cần tây nguyên chất, bột củ sen ngũ cốc dinh dưỡng,…xuất xứ tự nhiên không chứa chất bảo quản, không hóa chất độc hại và rất lành tính. Những nguyên liệu của Tiệm trà An Nhiên đều được tuyển chọn kỹ càng để có thể mang đến cho khách hàng trải những trải nghiệm tốt nhất!
❖ Sản phẩm: Củ sen sấy giòn An Nhiên 
❖ Thương hiệu: Tiệm trà An Nhiên 
❖ Số lượng: 3 gói, 5 gói 
❖ Xuất xứ: Việt Nam 
❖ Định dạng: gói 
❖ Thành phần: Củ sen, Dầu thực vật, Muối i-ốt, Đường, Gia vị 
❖ Hương vị: Phô mai, Trứng muối, Gà cay, Bơ tỏi, Rong biển 
❖ Hạn sử dụng: 6 tháng
CÔNG DỤNG CỦ SEN SẤY GIÒN 
– Củ Sen Sấy Giòn ăn giòn tan trong miệng

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.tiemtraannhien.vn/san-pham/tra-hoa/moc-tra-sinh-huyet/> (referer: None)
DEBUG:scrapy.core.scraper:Scraped from <200 https://www.tiemtraannhien.vn/san-pham/tra-hoa/moc-tra-sinh-huyet/>
{'url': 'https://www.tiemtraannhien.vn/san-pham/tra-hoa/moc-tra-sinh-huyet/', 'content': 'MỘC TRÀ SINH HUYẾT\nTiệm trà An Nhiên chuyên cung cấp các sản phẩm chăm sóc sắc đẹp và sức khỏe có thành phần từ thiên nhiên với công nghệ sản xuất khác biệt, trong đó có thể kể đến các sản phẩm như trà dưỡng tâm an thần, trà gừng đường nâu thảo mộc, trà dưỡng nhan, trà gạo lứt đậu đen, bột cần tây nguyên chất, bột củ sen ngũ cốc dinh dưỡng,…xuất xứ tự nhiên không chứa chất bảo quản, không hóa chất độc hại và rất lành tính. Những nguyên liệu của Tiệm trà An Nhiên đều được tuyển chọn kỹ càng để có thể mang đến cho khách hàng trải những trải nghiệm tốt nhất!\n❖ Sản phẩm: Mộc trà sinh huyết \n❖ Thương hiệu: Tiệm trà An Nhiên \n❖ Số lượng : 15 gói, 30 gói \n❖ Xuất x

Mộc Trà Sinh Huyết
====> h1_tag Mộc Trà Sinh Huyết
====>description MỘC TRÀ SINH HUYẾT
Tiệm trà An Nhiên chuyên cung cấp các sản phẩm chăm sóc sắc đẹp và sức khỏe có thành phần từ thiên nhiên với công nghệ sản xuất khác biệt, trong đó có thể kể đến các sản phẩm như trà dưỡng tâm an thần, trà gừng đường nâu thảo mộc, trà dưỡng nhan, trà gạo lứt đậu đen, bột cần tây nguyên chất, bột củ sen ngũ cốc dinh dưỡng,…xuất xứ tự nhiên không chứa chất bảo quản, không hóa chất độc hại và rất lành tính. Những nguyên liệu của Tiệm trà An Nhiên đều được tuyển chọn kỹ càng để có thể mang đến cho khách hàng trải những trải nghiệm tốt nhất!
❖ Sản phẩm: Mộc trà sinh huyết 
❖ Thương hiệu: Tiệm trà An Nhiên 
❖ Số lượng : 15 gói, 30 gói 
❖ Xuất xứ: Việt Nam 
❖ Định dạng: túi 
❖ Hạn sử dụng: 6 tháng
CÔNG DỤNG MỘC TRÀ SINH HUYẾT : 
Mộc Trà Sinh Huyết An Nhiên với những thành phần mang nét hương của 3 loại thảo dược tự nhiên: Hắc kỷ tử, Táo đỏ, Dâu tằm, Kỳ tử đỏ 
Mang đến lợi ích cho cơ thể: 
– Bổ sung máu, sắt

...

Trà Sả Lá Dứa
====> h1_tag Trà Sả Lá Dứa
====>description TRÀ SẢ LÁ DỨA AN NHIÊN DETOX THANH LỌC CƠ THỂ, ĐẸP DA GIẢM CÂN
Tiệm trà An Nhiên chuyên cung cấp các sản phẩm chăm sóc sắc đẹp và sức khỏe có thành phần từ thiên nhiên với công nghệ sản xuất khác biệt, trong đó có thể kể đến các sản phẩm như trà dưỡng tâm an thần, trà gừng đường nâu thảo mộc, trà dưỡng nhan, trà gạo lứt đậu đen, bột cần tây nguyên chất, bột củ sen ngũ cốc dinh dưỡng,…xuất xứ tự nhiên không chứa chất bảo quản, không hóa chất độc hại và rất lành tính. Những nguyên liệu của Tiệm trà An Nhiên đều được tuyển chọn kỹ càng để có thể mang đến cho khách hàng trải những trải nghiệm tốt nhất!
❖ Sản phẩm: Trà sả lá dứa An Nhiên 
❖ Thương hiệu: Tiệm trà An Nhiên 
❖ Số lượng: 30 gói 
❖ Xuất xứ: Việt Nam 
❖ Định dạng: túi trà 
❖ Thành phần: Sả, lá dứa 
❖ Hạn sử dụng: 6 tháng
CÔNG DỤNG TRÀ SẢ LÁ DỨA AN NHIÊN 
– Trà sả lá dứa giúp đẹp da, thải độc trong cơ thể, giảm cân an toàn. 
– Trà sả lá dứa giảm đau đầu và stress, t

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.tiemtraannhien.vn/san-pham/tra-thao-moc/tra-hoa-dau-biec-co-ngot/> (referer: None)
DEBUG:scrapy.core.scraper:Scraped from <200 https://www.tiemtraannhien.vn/san-pham/tra-thao-moc/tra-hoa-dau-biec-co-ngot/>
{'url': 'https://www.tiemtraannhien.vn/san-pham/tra-thao-moc/tra-hoa-dau-biec-co-ngot/', 'content': 'TRÀ ĐẬU BIẾC CỎ NGỌT AN NHIÊN DƯỠNG DA THẢI ĐỘC CƠ THỂ\nTiệm trà An Nhiên chuyên cung cấp các sản phẩm chăm sóc sắc đẹp và sức khỏe có thành phần từ thiên nhiên với công nghệ sản xuất khác biệt, trong đó có thể kể đến các sản phẩm như trà dưỡng tâm an thần, trà gừng đường nâu thảo mộc, trà dưỡng nhan, trà gạo lứt đậu đen, bột cần tây nguyên chất, bột củ sen ngũ cốc dinh dưỡng,…xuất xứ tự nhiên không chứa chất bảo quản, không hóa chất độc hại và rất lành tính. Những nguyên liệu của Tiệm trà An Nhiên đều được tuyển chọn kỹ càng để có thể mang đến cho khách hàng trải những trải nghiệm tốt nhất!\n❖ Sản phẩm: Trà đậu biếc cỏ ngọt An N

Trà Hoa Đậu Biếc Cỏ Ngọt
====> h1_tag Trà Hoa Đậu Biếc Cỏ Ngọt
====>description TRÀ ĐẬU BIẾC CỎ NGỌT AN NHIÊN DƯỠNG DA THẢI ĐỘC CƠ THỂ
Tiệm trà An Nhiên chuyên cung cấp các sản phẩm chăm sóc sắc đẹp và sức khỏe có thành phần từ thiên nhiên với công nghệ sản xuất khác biệt, trong đó có thể kể đến các sản phẩm như trà dưỡng tâm an thần, trà gừng đường nâu thảo mộc, trà dưỡng nhan, trà gạo lứt đậu đen, bột cần tây nguyên chất, bột củ sen ngũ cốc dinh dưỡng,…xuất xứ tự nhiên không chứa chất bảo quản, không hóa chất độc hại và rất lành tính. Những nguyên liệu của Tiệm trà An Nhiên đều được tuyển chọn kỹ càng để có thể mang đến cho khách hàng trải những trải nghiệm tốt nhất!
❖ Sản phẩm: Trà đậu biếc cỏ ngọt An Nhiên 
❖ Thương hiệu: Tiệm trà An Nhiên 
❖ Số lượng: 30 gói 
❖ Xuất xứ: Việt Nam 
❖ Định dạng: túi trà 
❖ Thành phần: Đậu biếc, cỏ ngọt 
❖ Hạn sử dụng: 6 tháng
CÔNG DỤNG TRÀ ĐẬU BIẾC CỎ NGỌT AN NHIÊN 
– Trà đậu biếc cỏ ngọt tăng hàm lượng collagen và độ đàn hồi của các tế bào da, giữ d

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.tiemtraannhien.vn/san-pham/tra-thao-moc/tra-hoa-thao-moc-duong-phen-vien/> (referer: None)
DEBUG:scrapy.core.scraper:Scraped from <200 https://www.tiemtraannhien.vn/san-pham/tra-thao-moc/tra-hoa-thao-moc-duong-phen-vien/>
{'url': 'https://www.tiemtraannhien.vn/san-pham/tra-thao-moc/tra-hoa-thao-moc-duong-phen-vien/', 'content': 'TRÀ HOA THẢO MỘC ĐƯỜNG ĐEN – ĐƯỜNG PHÈN MẬT ONG AN NHIÊN\nTiệm trà An Nhiên chuyên cung cấp các sản phẩm chăm sóc sắc đẹp và sức khỏe có thành phần từ thiên nhiên với công nghệ sản xuất khác biệt, trong đó có thể kể đến các sản phẩm như trà dưỡng tâm an thần, trà gừng đường nâu thảo mộc, trà dưỡng nhan, trà gạo lứt đậu đen, bột cần tây nguyên chất, bột củ sen ngũ cốc dinh dưỡng,…xuất xứ tự nhiên không chứa chất bảo quản, không hóa chất độc hại và rất lành tính. Những nguyên liệu của Tiệm trà An Nhiên đều được tuyển chọn kỹ càng để có thể mang đến cho khách hàng trải những trải nghiệm tốt nhất!\n❖ Sản phẩm:

Trà Hoa Thảo Mộc Đường Phèn Viên
====> h1_tag Trà Hoa Thảo Mộc Đường Phèn Viên
====>description TRÀ HOA THẢO MỘC ĐƯỜNG ĐEN – ĐƯỜNG PHÈN MẬT ONG AN NHIÊN
Tiệm trà An Nhiên chuyên cung cấp các sản phẩm chăm sóc sắc đẹp và sức khỏe có thành phần từ thiên nhiên với công nghệ sản xuất khác biệt, trong đó có thể kể đến các sản phẩm như trà dưỡng tâm an thần, trà gừng đường nâu thảo mộc, trà dưỡng nhan, trà gạo lứt đậu đen, bột cần tây nguyên chất, bột củ sen ngũ cốc dinh dưỡng,…xuất xứ tự nhiên không chứa chất bảo quản, không hóa chất độc hại và rất lành tính. Những nguyên liệu của Tiệm trà An Nhiên đều được tuyển chọn kỹ càng để có thể mang đến cho khách hàng trải những trải nghiệm tốt nhất!
❖ Sản phẩm: Trà đường phèn thảo mộc An Nhiên 
❖ Thương hiệu: Tiệm trà An Nhiên 
❖ Số lượng: 1 viên 
❖ Xuất xứ: Việt Nam 
❖ Định dạng: viên 
❖ Hạn sử dụng: 6 tháng
CÔNG DỤNG TRÀ ĐƯỜNG PHÈN THẢO MỘC AN NHIÊN 
– Trà đường phèn thảo mộc cải thiện vóc dáng, giảm cân an toàn 
– Trà đường phèn thảo mộc giúp tă

# Data Preprocessing

In [4]:
import json

# Load the `output.json` file
with open('output.json', 'r',encoding="utf8") as f:
    data = json.load(f)

In [5]:
len(data)

47

In [6]:
data[0]['content']

'CỦ SEN SẤY GIÒN – TRỌN VỊ NGON NÔNG SẢN VIỆT\nTiệm trà An Nhiên chuyên cung cấp các sản phẩm chăm sóc sắc đẹp và sức khỏe có thành phần từ thiên nhiên với công nghệ sản xuất khác biệt, trong đó có thể kể đến các sản phẩm như trà dưỡng tâm an thần, trà gừng đường nâu thảo mộc, trà dưỡng nhan, trà gạo lứt đậu đen, bột cần tây nguyên chất, bột củ sen ngũ cốc dinh dưỡng,…xuất xứ tự nhiên không chứa chất bảo quản, không hóa chất độc hại và rất lành tính. Những nguyên liệu của Tiệm trà An Nhiên đều được tuyển chọn kỹ càng để có thể mang đến cho khách hàng trải những trải nghiệm tốt nhất!\n❖ Sản phẩm: Củ sen sấy giòn An Nhiên \n❖ Thương hiệu: Tiệm trà An Nhiên \n❖ Số lượng: 3 gói, 5 gói \n❖ Xuất xứ: Việt Nam \n❖ Định dạng: gói \n❖ Thành phần: Củ sen, Dầu thực vật, Muối i-ốt, Đường, Gia vị \n❖ Hương vị: Phô mai, Trứng muối, Gà cay, Bơ tỏi, Rong biển \n❖ Hạn sử dụng: 6 tháng\nCÔNG DỤNG CỦ SEN SẤY GIÒN \n– Củ Sen Sấy Giòn ăn giòn tan trong miệng, thơm ngon, ngọt tràn cảm xúc \n– Củ Sen Sấy Giòn

In [7]:
#remove excess characters and duplicate text
import re
for item in data:
  item['content'] = re.sub(r'[^\S ]+', ' ', item['content'])

In [8]:
introduction = "Tiệm trà An Nhiên chuyên cung cấp các sản phẩm chăm sóc sắc đẹp và sức khỏe có thành phần từ thiên nhiên với công nghệ sản xuất khác biệt, trong đó có thể kể đến các sản phẩm như trà dưỡng tâm an thần, trà gừng đường nâu thảo mộc, trà dưỡng nhan, trà gạo lứt đậu đen, bột cần tây nguyên chất, bột củ sen ngũ cốc dinh dưỡng,…xuất xứ tự nhiên không chứa chất bảo quản, không hóa chất độc hại và rất lành tính. Những nguyên liệu của Tiệm trà An Nhiên đều được tuyển chọn kỹ càng để có thể mang đến cho khách hàng trải những trải nghiệm tốt nhất!"

In [9]:
count = 0
exception = []
for item in data:
  if introduction in item['content']:
    count += 1
    item['content'] = item['content'].replace(introduction, '')
  else:
    exception.append(item)
print(count)

44


In [10]:
warranty = "TIỆM TRÀ AN NHIÊN – NUÔI DƯỠNG VẺ ĐẸP TỪ SÂU BÊN TRONG  ⚫ Cam kết 100% nguyên liệu từ thiên nhiên, không chất phụ gia, chất bảo quản, chất hóa học.  ⚫ Sản phẩm có giấy tờ chứng nhận đầy đủ  ⚫ Nhiệt tình hỗ trợ, tư vấn khách hàng 24/7.  ⚫ Miễn phí trả hàng 07 ngày theo quy định của Shopee. QUY ĐỊNH HỖ TRỢ ĐỔI TRẢ HÀNG CỦA TIỆM TRÀ AN NHIÊN:  ❶ Điều kiện áp dụng (trong vòng 07 ngày kể từ khi nhận sản phẩm)  – Hàng hoá vẫn còn mới, chưa qua sử dụng  – Hàng hoá bị lỗi hoặc hư hỏng do vận chuyển hoặc do nhà sản xuất ❷ Trường hợp được chấp nhận:  – Hàng không đúng loại như quý khách đặt hàng  – Không đủ số lượng như trong đơn hàng ❸ Trường hợp không đủ điều kiện áp dụng chính sách:  – Quá 07 ngày kể từ khi Quý khách nhận hàng  – Gửi lại hàng không đúng mẫu mã, không phải sản phẩm của Tiệm trà An Nhiên"

In [11]:
count = 0
exception = []
for item in data:
  if warranty in item['content']:
    count += 1
    item['content'] = item['content'].replace(warranty, '')
  else:
    exception.append(item)
print(count)

40


# Convert all the records into LangChain's Document format

In [12]:
from uuid import uuid4
from bs4 import BeautifulSoup
from langchain_core.documents import Document

In [13]:
list_of_documents = []
for item in data:
  # Extract the price value from HTML tag
  soup = BeautifulSoup(item['price'], 'html.parser')
  price = soup.bdi.text.strip()
  content = item['title'] + ' - Giá tiền: ' + price + ' - ' + item['content']
  # Convert the records to LangChain's Document format and append them to a list
  list_of_documents.append(Document(page_content=content, metadata={"source": item['url'],
                                                                    "image_urls":item['image_urls']}))

In [14]:
#append introduction and warranty
list_of_documents.append(Document(page_content=introduction))
list_of_documents.append(Document(page_content=warranty))

In [15]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(list_of_documents))]

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key=gemini_token)

In [17]:
# Initialize the FAISS vector store
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

DEBUG:faiss.loader:Environment variable FAISS_OPT_LEVEL is not set, so let's pick the instruction set according to the current CPU
INFO:faiss.loader:Loading faiss with AVX512 support.
2025-03-24 14:09:12 [faiss.loader] INFO: Loading faiss with AVX512 support.
INFO:faiss.loader:Successfully loaded faiss with AVX512 support.
2025-03-24 14:09:12 [faiss.loader] INFO: Successfully loaded faiss with AVX512 support.
INFO:faiss:Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.
2025-03-24 14:09:12 [faiss] INFO: Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.


In [18]:
vector_store.add_documents(documents=list_of_documents, ids=uuids)

['6e95d53d-3525-4dcd-9d33-42240ffa877b',
 'd7d3b217-bad6-476d-9523-01b947ece8ad',
 '096e74d4-da11-4322-b821-c8ad26e33c22',
 '75290e31-223c-4fee-b635-eb2173a2383a',
 'e9768881-b639-4453-96e7-bead9446f89f',
 'b2b98226-7966-4735-90e8-fd51907952a1',
 '207601ef-12ef-4aa1-946f-b09c5aa1895c',
 '9d45a672-370b-453d-90e8-c6463d8452eb',
 '9adc7635-8dc2-4208-880a-8198e3c123e7',
 'e930c25c-dcf8-4317-960c-0e0b0312aa55',
 'f8585953-f3ec-40a1-b392-70716548c615',
 'd1b795bd-79f1-4e95-900f-f94a876a4982',
 '36852852-091c-451c-94b5-4f02fdd2b349',
 '750593b7-9651-46df-9716-4896e61517d0',
 '80808a89-f99c-4170-bc23-f26519597766',
 '9b236400-eb49-4be2-a6cb-7c56241029d1',
 '21567b64-9314-4cb0-81f0-23f8390f591b',
 '7e12f938-5b73-4091-bc0e-716d190c2d03',
 '3c46c497-c732-4bd3-a673-2cc5f323bde9',
 'fb2a7ed8-e7bb-431d-83b7-ebb1e9074e55',
 '36be28ed-9c80-4d19-b0e8-df526bbba510',
 'ba00c4fd-3505-4ca6-8292-66bc8908fddf',
 'd33d6d31-bfab-43e7-a7b6-9d28a77a34fd',
 '375702c5-6e8e-43dc-aa9d-d64bfbdb3ebd',
 '044463ec-d008-

In [19]:
# Similarity search example with the vector store
semantic_results = vector_store.similarity_search(
    "Cho tôi một loại trà có chiết xuất từ hoa hồng",
    k=10,
)
for res in semantic_results:
    print(f"* {res.page_content} [{res.metadata}]")

* Tiệm trà An Nhiên chuyên cung cấp các sản phẩm chăm sóc sắc đẹp và sức khỏe có thành phần từ thiên nhiên với công nghệ sản xuất khác biệt, trong đó có thể kể đến các sản phẩm như trà dưỡng tâm an thần, trà gừng đường nâu thảo mộc, trà dưỡng nhan, trà gạo lứt đậu đen, bột cần tây nguyên chất, bột củ sen ngũ cốc dinh dưỡng,…xuất xứ tự nhiên không chứa chất bảo quản, không hóa chất độc hại và rất lành tính. Những nguyên liệu của Tiệm trà An Nhiên đều được tuyển chọn kỹ càng để có thể mang đến cho khách hàng trải những trải nghiệm tốt nhất! [{}]
* TIỆM TRÀ AN NHIÊN – NUÔI DƯỠNG VẺ ĐẸP TỪ SÂU BÊN TRONG  ⚫ Cam kết 100% nguyên liệu từ thiên nhiên, không chất phụ gia, chất bảo quản, chất hóa học.  ⚫ Sản phẩm có giấy tờ chứng nhận đầy đủ  ⚫ Nhiệt tình hỗ trợ, tư vấn khách hàng 24/7.  ⚫ Miễn phí trả hàng 07 ngày theo quy định của Shopee. QUY ĐỊNH HỖ TRỢ ĐỔI TRẢ HÀNG CỦA TIỆM TRÀ AN NHIÊN:  ❶ Điều kiện áp dụng (trong vòng 07 ngày kể từ khi nhận sản phẩm)  – Hàng hoá 

In [20]:
# Using BM25 retriever from LangChain
from langchain_community.retrievers import BM25Retriever

In [21]:
bm25_retriever = BM25Retriever.from_documents(
  list_of_documents, k = 10
)

In [22]:
# Example with BM25 retriever
bm25_results = bm25_retriever.invoke("Cho tôi một loại trà có chiết xuất từ hoa hồng")
for res in bm25_results:
    print(f"* {res.page_content} [{res.metadata}]")

* Trà Đường Nâu Cam Quế An Nhiên - Giá tiền: 0₫ - TRÀ  ĐƯỜNG NÂU CAM QUẾ THẢO MỘC – Giảm đau bụng ngày dâu Trà đường nâu cam quế là thức uống bổ dưỡng từ đường nâu (mật mía) nguyên chất kết hợp với các loại thảo mộc: cam lát, nho khô, quế thanh, mộc quế hoa,… mang lại nhiều công dụng tuyệt vời cho sức khỏe.  Đường nâu cũng không chứa nhiều calo, không quá ngọt, thơm, ngọt ngào và có một hương vị tuyệt vời từ thảo mộ như cam lát, nho khô, quế thanh, hoa quế tăng hương vị . ❖ Sản phẩm: Trà đường nâu cam quế  ❖ Thương hiệu: Tiệm trà An Nhiên  ❖ Số lượng: 10 gói và 20 gói  ❖ Xuất xứ: Việt Nam  ❖ Định dạng: túi  ❖ Thành phần: Đường nâu, Cam Lát, Nho sấy, Thanh quế, Mộc quế hoa  ❖ Hạn sử dụng: 6 tháng CÔNG DỤNG :  – Làm cho da hồng hào, sắc mặt tươi trẻ hơn. Đặc biệt là với phụ nữ sau kỳ kinh nguyệt, sẽ làm cho làm ấm tử cung, giúp giảm đau bụng do kinh nguyệt.  – Bổ khí, bổ máu, rất tốt với người thiếu máu  – Làm dịu dạ dày, thúc đẩy trao đổi chất và làm sạch mạch máu  – Giảm stress, tốt ch

In [23]:
# Get the result chunks from BM25 Retriever and FAISS vector search
content = set()
retrieval_docs = []

for result in semantic_results:
  if result.page_content not in content:
    content.add(result.page_content)
    retrieval_docs.append(result)

for result in bm25_results:
  if result.page_content not in content:
    content.add(result.page_content)
    retrieval_docs.append(result)

len(retrieval_docs)

16

In [24]:
for res in retrieval_docs:
    print(f"* {res.page_content} [{res.metadata}]")

* Tiệm trà An Nhiên chuyên cung cấp các sản phẩm chăm sóc sắc đẹp và sức khỏe có thành phần từ thiên nhiên với công nghệ sản xuất khác biệt, trong đó có thể kể đến các sản phẩm như trà dưỡng tâm an thần, trà gừng đường nâu thảo mộc, trà dưỡng nhan, trà gạo lứt đậu đen, bột cần tây nguyên chất, bột củ sen ngũ cốc dinh dưỡng,…xuất xứ tự nhiên không chứa chất bảo quản, không hóa chất độc hại và rất lành tính. Những nguyên liệu của Tiệm trà An Nhiên đều được tuyển chọn kỹ càng để có thể mang đến cho khách hàng trải những trải nghiệm tốt nhất! [{}]
* TIỆM TRÀ AN NHIÊN – NUÔI DƯỠNG VẺ ĐẸP TỪ SÂU BÊN TRONG  ⚫ Cam kết 100% nguyên liệu từ thiên nhiên, không chất phụ gia, chất bảo quản, chất hóa học.  ⚫ Sản phẩm có giấy tờ chứng nhận đầy đủ  ⚫ Nhiệt tình hỗ trợ, tư vấn khách hàng 24/7.  ⚫ Miễn phí trả hàng 07 ngày theo quy định của Shopee. QUY ĐỊNH HỖ TRỢ ĐỔI TRẢ HÀNG CỦA TIỆM TRÀ AN NHIÊN:  ❶ Điều kiện áp dụng (trong vòng 07 ngày kể từ khi nhận sản phẩm)  – Hàng hoá 

In [25]:
# Use the BAAI/bge-reranker-v2-m3 model to rerank the order of the result chunks based on relevance score
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation
pairs = [["Cho tôi một loại trà có chiết xuất từ hoa hồng",doc.page_content] for doc in retrieval_docs]
score = reranker.compute_score(pairs,normalize = True)
score

DEBUG:pydot:pydot initializing
DEBUG:pydot:pydot 3.0.4
DEBUG:pydot.dot_parser:pydot dot_parser module initializing
DEBUG:pydot.core:pydot core module initializing
INFO:datasets:PyTorch version 2.6.0+cu124 available.
2025-03-24 14:09:26 [datasets] INFO: PyTorch version 2.6.0+cu124 available.
INFO:datasets:Polars version 1.21.0 available.
2025-03-24 14:09:26 [datasets] INFO: Polars version 1.21.0 available.
INFO:datasets:Duckdb version 1.2.1 available.
2025-03-24 14:09:26 [datasets] INFO: Duckdb version 1.2.1 available.
INFO:datasets:TensorFlow version 2.18.0 available.
2025-03-24 14:09:26 [datasets] INFO: TensorFlow version 2.18.0 available.
INFO:datasets:JAX version 0.5.2 available.
2025-03-24 14:09:26 [datasets] INFO: JAX version 0.5.2 available.
Level 1:tensorflow:Registering Relayout (<function _relayout_gradient at 0x7c3f2090ff60>) in gradient.
Level 1:tensorflow:Registering RelayoutLike (<function _relayout_grad_gradient at 0x7c3f20928180>) in gradient.
Level 1:tensorflow:Register

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-reranker-v2-m3/resolve/main/sentencepiece.bpe.model HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cdn-lfs-us-1.hf.co:443
DEBUG:urllib3.connectionpool:https://cdn-lfs-us-1.hf.co:443 "GET /repos/fb/3f/fb3fe87288ff7667a58d0dac0de359a1ea242a8cb977a35823603c68613285dc/cfc8146abe2a0488e9e2a0c56de7952f7c11ab059eca145a0a727afce0db2865?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27sentencepiece.bpe.model%3B+filename%3D%22sentencepiece.bpe.model%22%3B&Expires=1742828977&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MjgyODk3N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2ZiLzNmL2ZiM2ZlODcyODhmZjc2NjdhNThkMGRhYzBkZTM1OWExZWEyNDJhOGNiOTc3YTM1ODIzNjAzYzY4NjEzMjg1ZGMvY2ZjODE0NmFiZTJhMDQ4OGU5ZTJhMGM1NmRlNzk1MmY3YzExYWIwNTllY2ExNDVhMGE3MjdhZmNlMGRiMjg2NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=ob8Ox9M4dsH

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-reranker-v2-m3/resolve/main/tokenizer.json HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:https://cdn-lfs-us-1.hf.co:443 "GET /repos/fb/3f/fb3fe87288ff7667a58d0dac0de359a1ea242a8cb977a35823603c68613285dc/69564b696052886ed0ac63fa393e928384e0f8caada38c1f4864a9bfbf379c15?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tokenizer.json%3B+filename%3D%22tokenizer.json%22%3B&response-content-type=application%2Fjson&Expires=1742828295&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MjgyODI5NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2ZiLzNmL2ZiM2ZlODcyODhmZjc2NjdhNThkMGRhYzBkZTM1OWExZWEyNDJhOGNiOTc3YTM1ODIzNjAzYzY4NjEzMjg1ZGMvNjk1NjRiNjk2MDUyODg2ZWQwYWM2M2ZhMzkzZTkyODM4NGUwZjhjYWFkYTM4YzFmNDg2NGE5YmZiZjM3OWMxNT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVudC10eXBlPSoifV19&Signature=OyarionkGnAfWEHdsDd-Arqu-gvnSkmuz8arApzaAyO9VLzJuqjx

...

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-reranker-v2-m3/resolve/main/adapter_config.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-reranker-v2-m3/resolve/main/model.safetensors HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:https://cdn-lfs-us-1.hf.co:443 "GET /repos/fb/3f/fb3fe87288ff7667a58d0dac0de359a1ea242a8cb977a35823603c68613285dc/d9e3e081faff1eefb84019509b2f5558fd74c1a05a2c7db22f74174fcedb5286?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1742828292&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MjgyODI5Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2ZiLzNmL2ZiM2ZlODcyODhmZjc2NjdhNThkMGRhYzBkZTM1OWExZWEyNDJhOGNiOTc3YTM1ODIzNjAzYzY4NjEzMjg1ZGMvZDllM2UwODFmYWZmMWVlZmI4NDAxOTUwOWIyZjU1NThmZDc0YzFhMDVhMmM3ZGIyMmY3NDE3NGZjZWRiNTI4Nj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[0.00375078888895543,
 0.006413163741325347,
 0.00455611371445455,
 0.0037362205834114964,
 0.0072890775835919245,
 0.011642224164666004,
 0.0021489492836254285,
 0.0016875096190311083,
 0.001874861720261248,
 0.005797401254434213,
 0.003111508539858309,
 0.0028783480347663727,
 0.0008295891883591028,
 0.4041546398945171,
 0.0028118586602633557,
 0.04320403592344244]

In [26]:
# Put all the methods above into a class called 'Retriever'

class Retriever:
  def __init__(self, semantic_retriever, bm25_retriever, reranker):
    self.semantic_retriever = semantic_retriever
    self.bm25_retriever = bm25_retriever
    self.reranker = reranker

  def __call__(self,query):
    semantic_results = self.semantic_retriever.similarity_search(
      query,
      k=10,
    )
    bm25_results = self.bm25_retriever.invoke(query)

    content = set()
    retrieval_docs = []

    for result in semantic_results:
      if result.page_content not in content:
        content.add(result.page_content)
        retrieval_docs.append(result)

    for result in bm25_results:
      if result.page_content not in content:
        content.add(result.page_content)
        retrieval_docs.append(result)

    pairs = [[query,doc.page_content] for doc in retrieval_docs]

    scores = self.reranker.compute_score(pairs,normalize = True)

    # Retrieve the parent document from the child chunk based on a threshold score.
    context_1 = []
    context_2 = []
    context = []
    parent_ids = set()
    for i in range(len(retrieval_docs)):
      # Relevance score >= 0.6 will be used as context type 1 (indicating higher relevance to the query)
      if scores[i] >= 0.6:
        context_1.append(dict(retrieval_docs[i])['page_content'])
      # Relevance score >= 0.1 will be used as context type 2 (indicating medium to lower relevance to the query)
      elif scores[i] >= 0.1:
        context_2.append(dict(retrieval_docs[i])['page_content'])
      # If the relevance score < 0.1, it indicates that there are no relevant documents.
    if len(context_1) > 0:
      print('Context 1')
      context=context_1
    elif len(context_2) > 0:
      print('Context 2')
      context=context_2
    else:
      print('No relevant context')
    return context

In [27]:
# Test the Retriever
retriever = Retriever(semantic_retriever = vector_store, bm25_retriever = bm25_retriever, reranker = reranker)
context = retriever("Hoa hồng")
context

Context 2


['Trà Hoa Hồng Xí Muội Ô Mai - Giá tiền: 0₫ - TRÀ HOA HỒNG XÍ MUỘI Ô MAI AN NHIÊN ĐẸP DÁNG, MỜ THÂM, SÁNG DA  ❖ Sản phẩm: Trà hoa hồng xí muội ô mai An Nhiên  ❖ Thương hiệu: Tiệm trà An Nhiên  ❖ Số lượng: 30 gói  ❖ Xuất xứ: Việt Nam  ❖ Định dạng: túi  ❖ Thành phần: Ô mai, sơn trà, trần bì, dâu tằm, hoa hồng.  ❖ Hạn sử dụng: 6 tháng CÔNG DỤNG TRÀ HOA HỒNG XÍ MUỘI Ô MAI AN NHIÊN  – Trà hoa hồng xí muội ô mai giúp thanh nhiệt, giải độc, mát gan.  – Trà hoa hồng xí muội ô mai hỗ trợ điều hòa nội tiết tố, làm đều kinh nguyệt  – Trà hoa hồng xí muội ô mai giúp tăng cường sức khoẻ.  ']

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AIMessage,HumanMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
answerModel = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=gemini_token, temperature = 0.5)

In [32]:
answerPrompt = PromptTemplate.from_template("""
     Bạn là chuyên viên tư vấn khách hàng của tiệm trà thảo mộc An Nhiên.
     Câu hỏi của khách hàng: {query}\nChat History: {chat_history}\nTrả lời câu hỏi dựa vào Chat History và các thông tin sản phẩm của cửa hàng dưới đây: {source_information}.
 """)

In [33]:
answerChain = answerPrompt | answerModel

In [35]:
query = 'Tư vấn cho tôi loại trà có nguyên liệu là củ sen'
source_information = retriever(query)
answerChain.invoke({"query":query, "source_information":source_information, "chat_history":None})

Context 2


AIMessage(content='Chào bạn, rất vui được tư vấn cho bạn!\n\nHiện tại, Tiệm trà An Nhiên có 2 sản phẩm có thành phần từ củ sen:\n\n1.  **Bột Củ Sen Mix Hạt Dinh Dưỡng Ít Béo An Nhiên:** Sản phẩm này là sự kết hợp giữa bột củ sen nguyên chất và các loại hạt dinh dưỡng như táo đỏ, hạt điều, nho khô, hạt bí, hạt hạnh nhân, hạt hướng dương, mộc quế hoa, đường phèn, mè đen, hạt chia, hoa hồng. Bột củ sen rất phù hợp cho người muốn có bữa ăn nhanh gọn, tiện lợi mà vẫn đủ chất dinh dưỡng, ít calo, hỗ trợ trị các triệu chứng hô hấp, giúp trẻ hóa làn da và ngăn ngừa viêm nhiễm.\n2.  **Củ Sen Sấy Giòn An Nhiên:** Sản phẩm này là củ sen được sấy giòn với nhiều hương vị hấp dẫn như phô mai, trứng muối, gà cay, bơ tỏi, rong biển. Củ sen sấy giòn không chỉ là món ăn vặt thơm ngon mà còn giàu giá trị dinh dưỡng, giúp thanh lọc, giải nhiệt cơ thể, tăng cường hệ miễn dịch và cải thiện khẩu vị.\n\nBạn quan tâm đến sản phẩm nào hơn ạ? Hoặc bạn muốn sử dụng củ sen với mục đích gì (ví dụ: ăn vặt, bổ sung d

# Expose API

In [37]:
import os
import json
from google.colab import userdata
from pyngrok import ngrok
from flask import Flask, jsonify, request
from flask_cors import CORS

In [ ]:

ngrok.set_auth_token(ngrok_token)

app = Flask(__name__)
CORS(app)  # Apply CORS to the Flask app

@app.route('/v1/chat', methods=['POST'])
def chat_v1():
    # Extract parameters from the request
    user_message = request.json.get('message', {})
    chat_history = request.json.get('context', [])
    stream = True  # Default to False if not provided

    print(f'Message: {user_message}')
    print(f'Chat History: {chat_history}')

    context = retriever(user_message['human'])
    source_information = ""
    for doc in context:
      # content = dict(doc)['page_content'] + ' - Link ảnh: #' + str(doc.metadata['image_urls'])
      content = doc
      source_information+= content + "\n"
    if stream:
      def generate():
        for chunk in answerChain.stream({"query": user_message['human'], "chat_history": chat_history,"source_information": source_information}):
          yield chunk.content
      return app.response_class(generate(), mimetype='text/plain')
    else:
      reponse = answerChain.invoke({"query": user_message['human'], "chat_history":chat_history,"source_information": source_information})
      return jsonify({'response': reponse.content})

if __name__ == '__main__':
    # Start ngrok to tunnel the Flask app
    url = ngrok.connect(5000)
    print(f" * ngrok tunnel: {url}")

    # Start Flask app
    app.run(port=5000)

DEBUG:pyngrok.installer:Installing ngrok v3 to /root/.config/ngrok/ngrok ...
DEBUG:pyngrok.installer:Platform to download: linux_x86_64


DEBUG:pyngrok.installer:Download ngrok from https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip ...


DEBUG:pyngrok.installer:Extracting ngrok binary from /tmp/ngrok-v3-stable-linux-amd64.zip to /root/.config/ngrok/ngrok ...


DEBUG:pyngrok.installer:Installing default ngrok config to /root/.config/ngrok/ngrok.yml ...
INFO:pyngrok.process:Updating authtoken for default "config_path" of "ngrok_path": /root/.config/ngrok/ngrok
2025-03-24 14:12:05 [pyngrok.process] INFO: Updating authtoken for default "config_path" of "ngrok_path": /root/.config/ngrok/ngrok
INFO:pyngrok.ngrok:Opening tunnel named: http-5000-6eeba9b7-9c6c-48b8-a3ab-5f73c2a82e8c
2025-03-24 14:12:05 [pyngrok.ngrok] INFO: Opening tunnel named: http-5000-6eeba9b7-9c6c-48b8-a3ab-5f73c2a82e8c
DEBUG:pyngrok.process:ngrok process starting with PID: 3699
INFO:pyngrok.process.ngrok:t=2025-03-24T14:12:05+0000 lvl=info msg="no configuration paths supplied"
2025-03-24 14:12:05 [pyngrok.process.ngrok] INFO: t=2025-03-24T14:12:05+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2025-03-24T14:12:05+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
2025-03-24 14:12:05 [pyngrok.proces

 * ngrok tunnel: NgrokTunnel: "https://dd90-34-125-186-68.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2025-03-24 14:12:05 [werkzeug] INFO: WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
2025-03-24 14:12:05 [werkzeug] INFO: Press CTRL+C to quit
INFO:pyngrok.process.ngrok:t=2025-03-24T14:13:02+0000 lvl=info msg="join connections" obj=join id=11c98065429f l=127.0.0.1:5000 r=42.113.131.6:2384
2025-03-24 14:13:02 [pyngrok.process.ngrok] INFO: t=2025-03-24T14:13:02+0000 lvl=info msg="join connections" obj=join id=11c98065429f l=127.0.0.1:5000 r=42.113.131.6:2384


Message: {'human': 'xin chao'}
Chat History: [{'role': 'user', 'content': 'xin chao'}]
No relevant context


INFO:werkzeug:127.0.0.1 - - [24/Mar/2025 14:13:03] "POST /v1/chat HTTP/1.1" 200 -
2025-03-24 14:13:03 [werkzeug] INFO: 127.0.0.1 - - [24/Mar/2025 14:13:03] "POST /v1/chat HTTP/1.1" 200 -
INFO:pyngrok.process.ngrok:t=2025-03-24T14:13:19+0000 lvl=info msg="join connections" obj=join id=f58178655dad l=127.0.0.1:5000 r=42.113.131.6:2139
2025-03-24 14:13:19 [pyngrok.process.ngrok] INFO: t=2025-03-24T14:13:19+0000 lvl=info msg="join connections" obj=join id=f58178655dad l=127.0.0.1:5000 r=42.113.131.6:2139


Message: {'human': 'hiện tại tôi đang bị mất ngủ'}
Chat History: [{'role': 'user', 'content': 'xin chao'}, {'role': 'assistant', 'content': 'Chào bạn ạ! An Nhiên xin chào bạn. Bạn có cần An Nhiên tư vấn gì không ạ?'}, {'role': 'user', 'content': 'hiện tại tôi đang bị mất ngủ'}]
Context 2


INFO:werkzeug:127.0.0.1 - - [24/Mar/2025 14:13:20] "POST /v1/chat HTTP/1.1" 200 -
2025-03-24 14:13:20 [werkzeug] INFO: 127.0.0.1 - - [24/Mar/2025 14:13:20] "POST /v1/chat HTTP/1.1" 200 -
INFO:pyngrok.process.ngrok:t=2025-03-24T14:18:26+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
2025-03-24 14:18:26 [pyngrok.process.ngrok] INFO: t=2025-03-24T14:18:26+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
INFO:pyngrok.process.ngrok:t=2025-03-24T14:18:26+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
2025-03-24 14:18:26 [pyngrok.process.ngrok] INFO: t=2025-03-24T14:18:26+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
